In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation

This notebook grabs lobster exports and then illustrates what is going on with them

In [2]:
trade_type = "imports"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
crl = ["darkblue","slategray","crimson"]

background = "#ffffff"

In [4]:
last_month = 12

---
#### Grabe the trade data using the Census's API

In [5]:
def census_trade(url, trade_type, product_level):
    
    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m")
    # This is so I can call this correctly...
    
    if trade_type == "imports":
        
        trade_label = trade_type
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        df["import_quantity"] = df["CON_QY1_MO"].astype(float)
        
        df[product_level] = df["I_COMMODITY"].astype(str)
        
        df.drop(["CON_VAL_MO", "I_COMMODITY", "COMM_LVL","CON_QY1_MO"], axis = 1, inplace = True)
        
    if trade_type == "exports":
    
        trade_label = trade_type
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)

        df[product_level] = df["E_COMMODITY"].astype(str)        
        
        df.drop(["ALL_VAL_MO", "E_COMMODITY", "CTY_CODE", "COMM_LVL"], axis = 1, inplace = True)
    
    return df

---
### This is the Product Specific plot...

In [6]:
end_use = "hs?get=CON_VAL_MO,CTY_CODE,CTY_NAME,I_COMMODITY_LDESC,CON_QY1_MO,UNIT_QY1"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=HS10" + "&I_COMMODITY=760110*"

# This will grab all products that fall under the "non-alloyed unwrought aluminum"
# and will have the quantity values that all appear to be in Killograms

url = url

df = census_trade(url,"imports","hs6")

<Response [200]>


In [16]:
df.tail()

,CTY_CODE,CTY_NAME,I_COMMODITY_LDESC,UNIT_QY1,time,imports,import_quantity,hs6
4951,1220,CANADA,"UNWROUGHT ALUMINUM NOT ALLOYED, NESOI",KG,2020-05-01,219675367.0,131079712.0,7601106090
4952,1220,CANADA,"UNWROUGHT ALUMINUM NOT ALLOYED, NESOI",KG,2020-06-01,204768980.0,128515228.0,7601106090
4953,1220,CANADA,"UNWROUGHT ALUMINUM NOT ALLOYED, NESOI",KG,2020-07-01,191599352.0,112116384.0,7601106090
4954,1220,CANADA,"UNWROUGHT ALUMINUM NOT ALLOYED, NESOI",KG,2020-08-01,185788406.0,106866721.0,7601106090
4955,1220,CANADA,"UNWROUGHT ALUMINUM NOT ALLOYED, NESOI",KG,2020-09-01,96247492.0,54541693.0,7601106090


In [8]:
clist = ["CANADA"]

df = df[df.CTY_NAME.isin(clist)]

In [9]:
df.hs6.unique()

array(['7601103000', '7601106000', '7601106030', '7601106090'],
      dtype=object)

In [10]:
grp = df.groupby("time")

canada_df = grp.agg({"CTY_NAME": "first", "imports": "sum", "import_quantity": "sum"})

In [11]:
canada_df["import_tons"] = canada_df["import_quantity"]*0.00110231

In [12]:
canada_df["alu_price"] = canada_df["imports"]/canada_df["import_tons"]

In [13]:
canada_df.tail()

,CTY_NAME,imports,import_quantity,import_tons,alu_price
time,,,,,
2020-05-01,CANADA,310058018.0,184789112.0,203694.886049,1522.168887
2020-06-01,CANADA,306815830.0,190797856.0,210318.384647,1458.816026
2020-07-01,CANADA,260158707.0,153495205.0,169199.299424,1537.587377
2020-08-01,CANADA,224748629.0,128794731.0,141971.719929,1583.052097
2020-09-01,CANADA,116973918.0,66100807.0,72863.580564,1605.382512


In [14]:
def make_time_by_country(df, level):

    height = 533
    width = 600

    foobar = df[level]
    
    if level == "imports":
        
        title = "$ US Imports from CANADA (HS 760110, NON-ALLOYED UNWROUGHT ALUMINUM)"
        
    if level == "import_tons":
        
        title = "TONS of US Imports from CANADA (HS 760110, NON-ALLOYED UNWROUGHT ALUMINUM)"
        
    if level == "alu_price":
        
        title = "$ Price per TON of US Imports from CANADA (HS 760110, NON-ALLOYED UNWROUGHT ALUMINUM)"

    p = figure(plot_height=height, plot_width = width, x_axis_type="datetime",toolbar_location = 'below',
           tools = "box_zoom, reset, pan, xwheel_zoom", title = title, x_range = (dt.datetime(2015,1,1),dt.datetime(2020,last_month,31)) ) 

    numlines=1

    multi_line_source = ColumnDataSource({
        'xs': [foobar.index.values]*numlines,
        'ys': [foobar.values],
        'label': [df["CTY_NAME"].unique()],
        'color': ["crimson", "darkblue"]})

    p.multi_line(xs= "xs",
                ys= "ys",
                line_width=4, line_alpha=0.75, line_color = "color",
                 hover_line_alpha=0.75, hover_line_width = 4,
                hover_line_color= "color", source = multi_line_source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)
            
    TIMETOOLTIPS = """
            <div style="background-color:#F5F5F5; opacity: 0.95; border: 5px 5px 5px 5px;">
            <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold"> @label
             </span>
             </div>
             <div style = "text-align:left;">"""
    
    if level == "imports": 
    
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{$0.0a}</span>   
            </div>
            </div>
            """
        
    if level == "alu_price": 
    
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{$0.00a}</span>   
            </div>
            </div>
            """
        
    if level == "import_tons":
        
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{0.0a}</span>   
            </div>
            </div>
            """        
        
    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS,  line_policy='nearest', formatters={'$data_x': 'datetime'}))
    p.title.text_font_size = '13pt'
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75
    p.border_fill_color = background 
    
    
    tariffs232_box = BoxAnnotation(left=dt.datetime(2018,5,31), right=dt.datetime(2019,5,20), fill_color='slategray', fill_alpha=0.1)
    p.add_layout(tariffs232_box)
    
    if level == "import_tons":
        
        tariffs232_box1 = BoxAnnotation(top =  83000*(1+1.05), bottom = 83000, left=dt.datetime(2020,9,1), right=dt.datetime(2020,9,30), fill_color="darkblue", fill_alpha=0.1)
        p.add_layout(tariffs232_box1)
        
        tariffs232_box2 = BoxAnnotation(top =  70000*(1+1.05), bottom = 70000, left=dt.datetime(2020,10,1), right=dt.datetime(2020,10,31), fill_color="darkblue", fill_alpha=0.1)
        p.add_layout(tariffs232_box2)
        
        tariffs232_box3 = BoxAnnotation(top =  83000*(1+1.05), bottom = 83000, left=dt.datetime(2020,11,1), right=dt.datetime(2020,11,30), fill_color="darkblue", fill_alpha=0.1)
        p.add_layout(tariffs232_box3)
        
        tariffs232_box4 = BoxAnnotation(top =  70000*(1+1.05), bottom = 70000,left=dt.datetime(2020,12,1), right=dt.datetime(2020,12,31), fill_color="darkblue", fill_alpha=0.1)
        p.add_layout(tariffs232_box4)

    p.yaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = "13px"

    p.yaxis.minor_tick_line_color = None
    
    
    if level == "imports":
        p.yaxis.formatter = NumeralTickFormatter(format="($0. a)")
        
    if level == "alu_price":
        p.yaxis.formatter = NumeralTickFormatter(format="($0.0a)")
        
    if level == "import_tons":
        p.yaxis.formatter = NumeralTickFormatter(format="(0 a)")
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    
    p.toolbar.active_drag = None
    p.toolbar.autohide = True
    p.min_border_left = 0
    p.min_border_bottom = 0

    return p

In [15]:
qlevel = make_time_by_country(canada_df, "import_tons")
    
dlevel = make_time_by_country(canada_df, "imports")

plevel = make_time_by_country(canada_df, "alu_price")

tab3 = Panel(child= qlevel, title="Imports in Tons")

tab4 = Panel(child= dlevel, title="Imports in $")

tab5 = Panel(child= plevel, title="Price Per Ton in $")

output_file('.\\docs\\' + "us_alu_detailed.html")

div0 = Div(text = """Shaded grey area indicates when the Section 232 tariffs, under Proclamation 9704, on Canada went into effect
and were removed. Shaded blue area indicates date and ranges for which tariffs will not be 
reimposed. Hover over the line with your curser to view month by month values.""", max_width=555, background = background )
div0.sizing_mode= "scale_both"
        
outfig = column(Tabs(tabs=[tab3,  tab4, tab5], tabs_location = "above"), div0, sizing_mode="scale_both")

show(outfig)